# Data Science v cloude: Cesta "Azure ML SDK"

## Úvod

V tomto notebooku sa naučíme, ako používať Azure ML SDK na trénovanie, nasadenie a používanie modelu prostredníctvom Azure ML.

Predpoklady:
1. Vytvorili ste pracovný priestor Azure ML.
2. Načítali ste [dataset Heart Failure](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) do Azure ML.
3. Nahodili ste tento notebook do Azure ML Studio.

Ďalšie kroky sú:

1. Vytvoriť experiment v existujúcom pracovnom priestore.
2. Vytvoriť výpočtový klaster.
3. Načítať dataset.
4. Nastaviť AutoML pomocou AutoMLConfig.
5. Spustiť experiment AutoML.
6. Preskúmať výsledky a získať najlepší model.
7. Zaregistrovať najlepší model.
8. Nasadiť najlepší model.
9. Použiť endpoint.

## Špecifické importy pre Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Inicializácia pracovného priestoru  
Inicializujte objekt pracovného priestoru z uloženého nastavenia. Uistite sa, že konfiguračný súbor je prítomný v .\config.json  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Vytvorenie experimentu Azure ML

Vytvorme experiment s názvom 'aml-experiment' v pracovnom priestore, ktorý sme práve inicializovali.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Vytvorenie výpočtového klastru
Budete musieť vytvoriť [výpočtový cieľ](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) pre váš beh AutoML.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Dáta
Uistite sa, že ste nahrali dataset do Azure ML a že kľúč má rovnaký názov ako dataset.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Konfigurácia AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Spustenie AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Nasadenie najlepšieho modelu

Spustite nasledujúci kód na nasadenie najlepšieho modelu. Stav nasadenia môžete vidieť v Azure ML portáli. Tento krok môže trvať niekoľko minút.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Použitie koncového bodu
Môžete pridať vstupy do nasledujúcej ukážky vstupu.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nezodpovedáme za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
